In [2]:
import matplotlib.pyplot as plt
from dataload import *
from gan_architecture import *
from utils import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from dataload import *
from gan_architecture import *
import pandas as pd
import numpy as np
import copy

In [3]:
class SuperNetwork(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.lin = nn.Linear(input_size, 4)
        self.bn = nn.BatchNorm1d(4)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.lin(x)
        x = self.bn(x)
        x = self.activation(x)
        return x

In [4]:
bs = 16
input = torch.full((bs, 4), 1.5, dtype=torch.float, device='cpu')

In [5]:
s = SuperNetwork(4)
s2 = copy.deepcopy(s)
criterion = nn.BCELoss() 
optimizer = optim.Adam(s.parameters(), lr = 0.002)
optimizer2 = optim.Adam(s2.parameters(), lr = 0.002)

In [6]:
print(s.parameters)

<bound method Module.parameters of SuperNetwork(
  (lin): Linear(in_features=4, out_features=4, bias=True)
  (bn): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation): ReLU()
)>


In [7]:
loss = criterion(s(input), input)
loss2 = criterion(s2(input), input)
loss.backward()
loss2.backward()

In [8]:
for param in s.parameters():
  print(param.data)

for param in s2.parameters():
  print(param.data)

tensor([[-0.0378, -0.2988,  0.4105, -0.0871],
        [-0.1975, -0.0205, -0.2823, -0.1432],
        [ 0.4035,  0.3081, -0.2133, -0.3742],
        [ 0.3352, -0.0874, -0.1047,  0.0230]])
tensor([ 0.3113, -0.1266, -0.0317,  0.0631])
tensor([1., 1., 1., 1.])
tensor([0., 0., 0., 0.])
tensor([[-0.0378, -0.2988,  0.4105, -0.0871],
        [-0.1975, -0.0205, -0.2823, -0.1432],
        [ 0.4035,  0.3081, -0.2133, -0.3742],
        [ 0.3352, -0.0874, -0.1047,  0.0230]])
tensor([ 0.3113, -0.1266, -0.0317,  0.0631])
tensor([1., 1., 1., 1.])
tensor([0., 0., 0., 0.])


In [9]:
gen = s.modules()
next(gen)
m_list  = list(gen)

gen = s2.modules()
next(gen)
m_list2 = list(gen)


for l in m_list[:-1] :
    print(l.weight)

for l in m_list2[:-1] :
    print(l.weight)

Parameter containing:
tensor([[-0.0378, -0.2988,  0.4105, -0.0871],
        [-0.1975, -0.0205, -0.2823, -0.1432],
        [ 0.4035,  0.3081, -0.2133, -0.3742],
        [ 0.3352, -0.0874, -0.1047,  0.0230]], requires_grad=True)
Parameter containing:
tensor([1., 1., 1., 1.], requires_grad=True)
Parameter containing:
tensor([[-0.0378, -0.2988,  0.4105, -0.0871],
        [-0.1975, -0.0205, -0.2823, -0.1432],
        [ 0.4035,  0.3081, -0.2133, -0.3742],
        [ 0.3352, -0.0874, -0.1047,  0.0230]], requires_grad=True)
Parameter containing:
tensor([1., 1., 1., 1.], requires_grad=True)


In [11]:
apply_weight_decay(*s.modules(), weight_decay_factor=0.005, wo_bn=True)

In [12]:
optimizer.step()
optimizer2.step()

In [13]:
for l in m_list[:-1] :
    print(l.weight)

for l in m_list2[:-1] :
    print(l.weight)

Parameter containing:
tensor([[-0.0358, -0.2968,  0.4085, -0.0851],
        [-0.1955, -0.0185, -0.2803, -0.1412],
        [ 0.4015,  0.3061, -0.2113, -0.3722],
        [ 0.3332, -0.0854, -0.1027,  0.0210]], requires_grad=True)
Parameter containing:
tensor([1., 1., 1., 1.], requires_grad=True)
Parameter containing:
tensor([[-0.0378, -0.2988,  0.4105, -0.0871],
        [-0.1975, -0.0205, -0.2823, -0.1432],
        [ 0.4035,  0.3081, -0.2133, -0.3742],
        [ 0.3352, -0.0874, -0.1047,  0.0230]], requires_grad=True)
Parameter containing:
tensor([1., 1., 1., 1.], requires_grad=True)
